In [5]:
import os, sys
import time
import threading

In [6]:
def getFPS():
    result= []
    read_path = "/home/petalinux/.temp/pf_tx"
    rf = os.open(read_path, os.O_RDONLY)
    s = b''
    while True:
        try:
            s += os.read(rf, 1024)
            if(len(s) >= 1024):
                continue 

            print("received msg:",len(s))
            for string in s.split():
                info = str(string,encoding = "utf-8").split(',')
                if(info[0] == 'reportFPS'):
                    result.append(float(info[1]))
                
            break
        except NameError as error: 
            print(error) 
            break

    os.close(rf)
    os.remove(read_path)
    return result

In [7]:
getFPS()

received msg: 44


[30.562025]

In [8]:
import os, sys
import time
import threading
class kv260adpFPS():
    _instance_dict = {}
    busy = False
    def __new__(cls,read_path, *args, **kw):
        
        if(read_path in cls._instance_dict.keys()):
            print("warning:",read_path," already in program!")
            return cls._instance_dict[read_path]
        cls._instance_dict[read_path] = object.__new__(cls)
        return cls._instance_dict[read_path]    
    

    def __init__(self, read_path,len = 5):
        self.read_path = read_path
        self.len =len
        
    def get_bytesVersion(self,timeout = None):
        if(self.busy):
            print("busy")
            return
        
        self.result = []
        self.busy = True
        if(not  os.path.exists(self.read_path)):
            os.mkfifo(self.read_path)
        def readfifodata():
            self.rf = os.open(self.read_path, os.O_RDONLY)
            s = b''
            while True:
                s = os.read(self.rf, 1024)
                if(len(s) >= 1024):
                    continue 
#                 print("received msg:",len(s))
                
                for string in s.split():
            
                    fps = self.praseFPSStr(str(string,encoding = "utf-8"))
                    if(not fps== None):
                        self.result.append(fps)
                s = b''
                if(len(self.result)>= self.len):
                    break
                time.sleep(0.1)
                
            os.close(self.rf)
                
        t1 = threading.Thread(target=readfifodata,args=())
        t1.start()
        t1.join(timeout)
        os.remove(self.read_path)
        self.busy = False
        return self.result
    
    
    def praseFPSStr(self,msg):
        info = msg.split(',')
        if(info[0] == 'reportFPS'):
            try:
                fps = float(info[1])
            except:
                print("invaild data:",info[1])
                fps = None
        return fps
            
    
    def get_fileVersion(self,timeout = None):
        if(self.busy):
            print("busy")
            return
        self.results = []
        self.busy = True
        if(not os.path.exists(self.read_path)):
            os.mkfifo(self.read_path)
        def func():            
            fifo_read_fd  = open(self.read_path,'r') 
            maxcnt = self.len
            while(maxcnt>0):        
                line = fifo_read_fd.readline()
                if len(line) == 0:
                    time.sleep(0.1)
                    continue 
                maxcnt-=1
#                 print(line)
                self.results.append(self.praseFPSStr(line))
            fifo_read_fd.close()    
        t1 = threading.Thread(target=func,args=())
        t1.start()
        t1.join(timeout)
        os.remove(self.read_path)
        self.busy = False
        return self.results
        

        
        
        
        
        
# fps = kv260adpFPS("/home/petalinux/.temp/pf_tx",len=5)


# res = fps.get_fileVersion(3)
# print(res)
# res = fps.get_bytesVersion(3)
# print(res)



In [328]:
import os
import threading
import time


class FIFOSendObj():
    _instance_dict = {}
    busy = False
    timeout = 1
    
    def __new__(cls, write_path, *args, **kw):
        if(write_path in cls._instance_dict.keys()):
            return cls._instance_dict[write_path]
        cls._instance_dict[write_path] = object.__new__(cls)
        return cls._instance_dict[write_path]
        
    def __init__(self,write_path, timeout = 1, *args, **kw):
        self.write_path =  write_path
        self.timeout = timeout
        
    def writeData2FIFO(self,msg):
        if(self.busy):
            print("Bus is busy")
            return
        
        self.busy = True
        self.istimeout = True
        def writefifodata(write_path,msg):
            if(not os.path.exists(write_path)):
                os.mkfifo(write_path)
            fifo_write_fd = open(write_path, 'w', 1 )
            protect = open(write_path, 'r')
            fifo_write_fd.write(msg)  
            fifo_write_fd.flush()
            fifo_write_fd.close()
            time.sleep(0.2)
            protect.close()
            self.istimeout = False
#             print("write finished")
        
        self.t1 = threading.Thread(target=writefifodata,args=(self.write_path,msg))
        self.t1.start()
        self.t1.join(self.timeout)
        if(os.path.exists(self.write_path)):
            os.remove(self.write_path)
        self.busy = False
        if(self.istimeout):
            print("write fifo timeout!")
            return False
#             
        else:
            return True
            


class kv260adpModelCtr(object):
   
    modelname = ""
    modelpath = ""
    modelclass = ""
    enable_str = ["ON","1"]
    disable_str = ["OFF","0"]
    
    def __init__(self,write_path="",*args, **kw):
        self.write_path = write_path
        self.timeout = 1;
        
        
    def checkModelfile(self):
        if(self.modelname == "" or self.modelpath==""):
            assert self.write_path != "" ,"model file path unset"

        if(not os.path.exists(os.path.join(self.modelpath,self.modelname,self.modelname+'.xmodel'))):
            assert self.write_path != "" ,"xmodel file does not exist!"

        if(not os.path.exists(os.path.join(self.modelpath,self.modelname,self.modelname+'.prototxt'))):
            assert self.write_path != "" ,"prototxt file does not exist!"
        return True
    
    def checkModelClass(self):
        vaildclasses=['YOLOV3','FACEDETECT','CLASSIFICATION','SSD','REID','REFINEDET','TFSSD','YOLOV2','ROADLINE','SEGMENTATION']
        if(self.modelclass not in vaildclasses):
            assert self.write_path != "" ,"invalid class, please use valid calss name:{}".format(vaildclasses)
           
        return True
    
    def checkWritePath(self):
        assert self.write_path != "" ,"please set write path"
     
    def resetWritePath(self, write_path):
        if(write_path != ""):
            self.write_path = write_path;
        self.checkWritePath()
        
# =====================================================================================
    def setNewModel(self,modelname, modelclass, modelpath, write_path = ""):
        self.resetWritePath(write_path)
        header = "switch2model" 
        self.modelname = modelname
        self.modelpath = modelpath
        self.modelclass = modelclass
        self.checkModelfile()
        self.checkModelClass()
        cmd = "{},{},{},{}".format(header,self.modelname,self.modelclass,self.modelpath)
        fifosend = FIFOSendObj(self.write_path,self.timeout)
        fifosend.writeData2FIFO(cmd)      
        
    def setNewREIDModel(self,modelname,modelpath,write_path = ""):
        self.resetWritePath(write_path)
        header = "switch2reidmodel" 
        self.modelname = modelname
        self.modelpath = modelpath
        
        if(not self.checkModelfile()):
            return
        
        cmd = "{},{},{}".format(header,self.modelname,self.modelpath)
        fifosend = FIFOSendObj(self.write_path,self.timeout)
        fifosend.writeData2FIFO(cmd)
        
    def setDPUInvteral(self,inverteral,write_path = ""):
        self.resetWritePath(write_path)
        inverteral = int(inverteral)
        header = "pluginCtr_invteral" 
        if(inverteral<1):
            print("invalid value:",inverteral)
            return
        fifosend = FIFOSendObj(self.write_path,self.timeout)
        fifosend.writeData2FIFO('{},{}'.format(header,inverteral))
    
    
    def setDPUenable(self,enable,write_path = ""):
        self.resetWritePath(write_path)
        header = "pluginCtr_DPUenable" 
        
        
        if str(enable).upper() in self.enable_str:
            ctr = 1
        elif str(enable).upper() in self.disable_str:
            ctr = 0
        else:
            print("invaild",str(enable).upper(),)
            return
            
        fifosend = FIFOSendObj(self.write_path,self.timeout)
        fifosend.writeData2FIFO('{},{}'.format(header,ctr))
        
    def setIndicaterUI(self,on,write_path = ""):
        self.resetWritePath(write_path)
        
        if(on == "ON" or on == 1 or on == "on"):
            status = 1;
        else:
            status = 0; 
   
        
        header = "runindicater" 
        fifosend = FIFOSendObj(self.write_path,self.timeout)
        fifosend.writeData2FIFO('{},{}'.format(header,status))
        
    def getFPSfromFile(self, file):
        f = open(file)
        line = f.readline()
#         print("fps:",line)
        return float(str(line))
        
    def getSegmentationResult(self,file):
        f = open(file)
        line = f.readline()
        rescnt = []
        p_sum = 0
        for s in line.split(','):
            rescnt.append(int(s))
            p_sum += int(s)
        
        res_normal = [x/p_sum for x in rescnt]
    
        car_related = rescnt[13] + rescnt[14] + rescnt[15] + rescnt[17]
        people_related = rescnt[18] + rescnt[11] + rescnt[12]
               
        k1 = 0.1
        k2 = 0.1
        k3 = 0.2
        k4 = 1.5
        if(car_related < k1 and people_related < k2 ):
            if(rescnt[0]> k3):
                return 'car'
            else:
                return None
        
        
        if(car_related > people_related * 1.5):
            return 'car'
        else:
            return 'people'
    

    
def getSegRes_FFC():
        classification = {-1:"unknown",0:"people",1:"car",2:"road"}
        result = -1
        read_path = "/home/petalinux/.temp/segresults"
        rf = os.open(read_path, os.O_RDONLY)
        s = b''
        while True:
            try:
                s += os.read(rf, 1024)
                if(len(s) >= 1024):
                    continue 
    #             print("received msg:",len(s),s)
                for string in s.split():
                    info = str(string,encoding = "utf-8").split(',')
                    if(info[0] == 'reportSeg' and int(info[1]) in classification.keys()):
                        result = int(info[1])
                break
            except NameError as error: 
                print(error) 
                break

        os.close(rf)
        os.remove(read_path)
        return result
    
        
        
def reidSwtichModel():
    reid_models = [["carid","/opt/xilinx/share/vitis_ai_library/models/B3136/"],
                   ["personreid-res18_pt","/opt/xilinx/share/vitis_ai_library/models/kv260-aibox-reid/"],
                   ["RN18_08","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_06","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_04","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_02","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"]]
    reid_modelctr = kv260adpModelCtr("/home/petalinux/.temp/reidrx")
    reid_modelctr.setNewREIDModel(reid_models[5][0],reid_models[5][1])

def segmentationSwtichModel():
    modelctr = kv260adpModelCtr("/home/petalinux/.temp/dpu_seg_rx")
    modelctr.setNewModel("SemanticFPN_cityscapes_256_512","SEGMENTATION","/opt/xilinx/share/vitis_ai_library/models/B3136/")
    modelctr.setNewModel("ENet_cityscapes_pt","SEGMENTATION","/opt/xilinx/share/vitis_ai_library/models/B3136/")
    modelctr.setDPUInvteral(10)



    
    
def branchSwitch(maxcnt = -1):
    
    # READ
    FFC_SEG_RES = "/home/petalinux/.temp/segresults"
    FFC_FPS_HB = '/home/petalinux/.temp/pf_tx'
    FILE_FPS = '/home/petalinux/.temp/fps'
    FILE_FPS1 = '/home/petalinux/.temp/fps_branch1'
    FILE_FPS2 = '/home/petalinux/.temp/fps_branch2'
    FILE_SEGMENTATION = '/home/petalinux/.temp/segres'
    # WRITE
    FFC_UI_BRANCH1 = '/home/petalinux/.temp/runstatus1_rx'
    FFC_UI_BRANCH2 = '/home/petalinux/.temp/runstatus2_rx'  
    FFC_DPU_BRANCH_CAR_CTR = '/home/petalinux/.temp/dpu_yolo_rx'
    FFC_DPU_BRANCH_PEO_CTR ='/home/petalinux/.temp/dpu_refinedet_rx'
    FFC_DPU_SEG_CTR = '/home/petalinux/.temp/dpu_seg_rx'

    lastseg = -1
    traffic_modelctr = kv260adpModelCtr()
    traffic_modelctr.setDPUInvteral(30,FFC_DPU_SEG_CTR)
    while(maxcnt != 0):
#         fps1 = traffic_modelctr.getFPSfromFile(FILE_FPS1)
#         fps2 = traffic_modelctr.getFPSfromFile(FILE_FPS2)
        maxcnt -= 1
        seg = getSegRes_FFC()
#         seg = traffic_modelctr.getSegmentationResult(FILE_SEGMENTATION)
        if lastseg != seg:
            print("do switch",lastseg,seg)
            lastseg = seg
            if(seg in [0]):
                traffic_modelctr.setIndicaterUI('on',FFC_UI_BRANCH2)
                traffic_modelctr.setIndicaterUI('off',FFC_UI_BRANCH1)
                traffic_modelctr.setDPUenable('on',FFC_DPU_BRANCH_CAR_CTR)
                traffic_modelctr.setDPUenable('off',FFC_DPU_BRANCH_PEO_CTR)

            elif(seg in [1]):
                traffic_modelctr.setIndicaterUI('on',FFC_UI_BRANCH1)
                traffic_modelctr.setIndicaterUI('off',FFC_UI_BRANCH2)
                traffic_modelctr.setDPUenable('off',FFC_DPU_BRANCH_CAR_CTR)
                traffic_modelctr.setDPUenable('on',FFC_DPU_BRANCH_PEO_CTR)
   
        time.sleep(1) 
    
# traffic_modelctr.setDPUenable('0',FFC_DPU_BRANCH_PEO_CTR)
# traffic_modelctr.setDPUenable('on',FFC_DPU_BRANCH_CAR_CTR)
# traffic_modelctr.setDPUenable('on',FFC_DPU_BRANCH_CAR_CTR)
# traffic_modelctr.setDPUInvteral(30,FFC_DPU_SEG_CTR)
# branchSwitch()
# getSegRes_FFC()

In [ ]:
branchSwitch()

do switch -1 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do switch 1 0
do switch 0 1
do sw

In [338]:
# SET interval frame of AI model
branchSwitch(1)
modelctr = kv260adpModelCtr()
# YOLO
modelctr.setDPUInvteral(2,'/home/petalinux/.temp/dpu_yolo_rx')
# Segmentaion
modelctr.setDPUInvteral(10,'/home/petalinux/.temp/dpu_seg_rx')

do switch -1 0


In [222]:
def reidSwtichModel(i):
    reid_models = [["RN18_08","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_06","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_04","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                   ["RN18_02","/opt/xilinx/share/vitis_ai_library/models/B3136/carid/"],
                  ["personreid-res18_pt","/opt/xilinx/share/vitis_ai_library/models/kv260-aibox-reid/"]]
    reid_modelctr = kv260adpModelCtr("/home/petalinux/.temp/reidrx")
    reid_modelctr.setNewREIDModel(reid_models[i][0],reid_models[i][1])
reidSwtichModel(1)

In [329]:

modelctr = kv260adpModelCtr("/home/petalinux/.temp/dpu_seg_rx",timeout=2)
# modelctr.setNewModel("SemanticFPN_cityscapes_256_512","SEGMENTATION","/opt/xilinx/share/vitis_ai_library/models/B3136/")
# modelctr.setNewModel("ENet_cityscapes_pt","SEGMENTATION","/opt/xilinx/share/vitis_ai_library/models/B3136/")
modelctr.setDPUInvteral(1)
